In [1]:
import pandas as pd 
import numpy as np
from sklearn.cross_validation import train_test_split
from sklearn import linear_model
#import pylab as plt
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import statsmodels.api as sm
%matplotlib inline

###This notebook will focus on the analytical part of the 311 demographics analysis: that is, multilinear regressions between resident and working population socio-demographic attributes and  the number of 311 calls by type per capita, at the census tract level will be performed and analyzed

In [2]:
#Upload the working population attributes, and the resident population attributes at the CT level. Then, merging both datasets
#File with resident demographics
demographics_NTA_NYC_residents=pd.read_csv('data/demographics_nta_NYC_residents_compiled.csv').drop(['Unnamed: 0', 'Unnamed: 0.1'],axis=1)
#File with working population demographics
demographics_NTA_NYC_workers=pd.read_csv('data/demographics_nta_NYC_workers_compiled.csv').drop(['Unnamed: 0', 'Unnamed: 0.1'],axis=1)
#Merging both datasets:
demographics_NTA_NYC=pd.merge(demographics_NTA_NYC_residents,demographics_NTA_NYC_workers,on='Neighborhood',how='inner')
print len(demographics_NTA_NYC_residents),len(demographics_NTA_NYC_workers),len(demographics_NTA_NYC)

195 194 194


So, 'demographics_NTA_NYC' will be the dataset that combines working and resident populations for each NTA

In [3]:
residents_demographics=demographics_NTA_NYC_residents.columns[2:]
workers_demographics=demographics_NTA_NYC_workers.columns[2:]

In [4]:
#Upload the 311 calls by type per capita
calls_bytype_normalized=pd.read_csv('data/Call by type with normalization by resident - NTA level_v2.csv').drop('Unnamed: 0',axis=1)

####The next dataset will combine all the information (demographics + calls by type normalized by resident population)

In [5]:
#callsbytype_attributes will be a dataframe combining all the information (demographics + calls by type)
callsbytype_attributes=pd.merge(calls_bytype_normalized,demographics_NTA_NYC, on='Neighborhood',how='inner')
print len(callsbytype_attributes), len(calls_bytype_normalized),len(demographics_NTA_NYC)

190 190 194


In [6]:
#list of type of calls that will be the dependent variables in regressions
types_of_calls=calls_bytype_normalized.columns[:-3]  #types of calls

In [7]:
regressors=[u'Population under 18', u'population between 18 and 34',
       u'population between 35 to 64', u'population 65 and over',
       u'Population white', u'population black', u'Population asian',
       u'population hispanic', u'population other race',
       u'family households', u'nonfamily households',
       u'population education high school', u'population education bachelors',
       u'population education masters', u'population education phd',
        u'owner  occupied units',
       u'renter occupied units', u'transportation car', u'number of cars',
       u'transportation public', u'tranportation motorcycle',
        u'household income form 10 to 40',
       u'household income form 40 to 75', u'household income 75 and above',
       u'house value for 20 to 100', u'house value for 100 to 500',
       u'house value 500 or more', u'rent bewteen 300 and 1000',
       u'rent bewteen 1000 and 2000', u'rent 2000 or more',
       u'Transportation Other means', u'population between 18 and 34_n',
       u'population between 35 to 64_n', u'population 65 and over_n',
       u'Population white_n', u'population black_n', u'Population asian_n',
       u'population hispanic_n', u'population other _n',
       u'family households_n', u'nonfamily households_n',
       u'population education high school_n',
       u'population education bachelors_n', u'population education masters_n',
       u'population education phd_n', u'household income less than 10_n',
       u'owner  occupied units_n', u'renter occupied units_n',
       u'transportation car_n', u'transportation public_n',
       u'tranportation motorcycle_n', 
       u'household income form 10 to 40_n',
       u'household income form 40 to 75_n', u'household income 75 and above_n',
       u'house value for 20 to 100_n', u'house value for 100 to 500_n',
       u'house value 500 or more_n', u'rent bewteen 300 and 1000_n',
       u'rent bewteen 1000 and 2000_n', u'Transportation Other means_n']

Lets Consider different groups of population $g=1,2,…,n$ (based on our demographic indicators) and let:


$Pr(a,g)$ - the total number of residents in the location $a$ of group $g$ 

while $Pc(a,g)$ the number of commuters.
 
Let the unknown (subject to fit) complaining behavior be defined by the average number of complains of type $t$ per resident of group $g$ within his/her place of residency be $rc(g,t)$

Let also, $wc(g,t)$ be the number of complains of type $t$ per commuter of type $g$.

Then the total observed number of complains of type $t$ in the area $a$ is:

$$C(a,t)=\sum_{g,t} Pr(a,g) \ rc(g,t) + \sum_{g,t} Pc(a,g) \ wc(g,t) \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \  \text{(1)}$$ 

Then we know $Pr(a,g)$ and $Pc(a,g)$ (those are our regressors), we know the output variable $C(a,t)$ from 311 statistics. We need to fit the $rc(g,t)$, $wc(g,t)$ - slope coefficients of the multivariate linear regression.

This will give us complaining behavior per people of different groups and it will be distinguished by the complaining mode - while at home and while on the way.



###we will procced as follows:

STEP 1) Lasso regression:

Regressors:  

$Pr(a,g)$ - the total number of residents in the location $a$ of group $g$.
            
$Pc(a,g)$  number of commuters in the location a of group $g$

Target variable to be fit: $rc(g,t)$ -   average number of complains of type $t$ per resident of group $g$                                                      within his/her place of residency 

STEP 2) predict the number of complains per capita $wc(g,t)$ from the results of step 1, using equation $(1)$

Using the predicted value $rc(g,t)$ in each area, we are able to get a $wc(g,t)$ prediction (from the formula of the observed total calls by type $C(a,t)$ variable) 

In [8]:
#RANDOM FOREST

In [9]:
from sklearn.datasets import load_boston
from sklearn.ensemble import RandomForestRegressor

The next piece of code will prepare the datasets for the regressions: X the dataframe combining all the regressors, Y a column of the number of calls normalized.
The variable "typeof" is the type of call 

In [10]:
typeof= 'Noise'
print typeof
A2=np.append(np.append(regressors,typeof),'Neighborhood')   #selection of columns
myframe1=callsbytype_attributes[A2].dropna() 
X=myframe1[regressors]
Y=myframe1[typeof]

Noise


In [11]:
#parameters for random forest
max_features = ['auto','sqrt','log2']
criterion = ['gini','entropy']
n_estimators = [100,500,1000,5000]
import itertools
from sklearn.metrics import r2_score

In [12]:
#The code will just run if the type of complain is present in more than 50 NTA.
if len(myframe1)>50:
    #Divide the datasets in train, validation and test sets
    X_pre_train, X_test, label_pre_train, Y_test = train_test_split(X, Y, test_size=0.30, random_state=1)
    X_train, X_val, Y_train, Y_val = train_test_split(X_pre_train, label_pre_train, test_size=0.25, random_state=1)
    d=[]
    for mf,n_estim in itertools.product(max_features,n_estimators):
        forest=RandomForestRegressor(n_estimators=n_estim, max_depth=None, min_samples_split=2, min_samples_leaf=1,
         min_weight_fraction_leaf=0.0, max_features=mf,
         max_leaf_nodes=None, bootstrap=True, oob_score=False, n_jobs=1, random_state=None, verbose=0, warm_start=False)
        forest.fit(X_train,Y_train)
        forest_pred = forest.predict(X_val)
        name = '%s,%s'%(str(mf),str(n_estim))
        d.append((name,r2_score(Y_val,forest_pred)))
    #Take the best R2 over the validation set for selecting the 
    best_solution=sorted(d, key=lambda tup: tup[1])[-1]
    parameters= best_solution[0].split(',')    
    mf=parameters[0]
    n_estim=parameters[1]
    #Run the regression again with the selected parameters o
    forest=RandomForestRegressor(n_estimators=int(n_estim), max_depth=None, min_samples_split=2, min_samples_leaf=1,
         min_weight_fraction_leaf=0.0, max_features=mf,
         max_leaf_nodes=None, bootstrap=True, oob_score=False, n_jobs=1, random_state=None, verbose=0, warm_start=False)
    forest.fit(X_train,Y_train)
    features_scores_list=sorted(zip(map(lambda x: round(x, 4), forest.feature_importances_), regressors), 
             reverse=True)
    features_scores_table=pd.DataFrame(features_scores_list, columns=['slope_coeff', 'variable'])
    features_scores_table

#The same previous code but for all the types of complains

In [ ]:
results={}
for typeof in types_of_calls:
    A2=np.append(np.append(regressors,typeof),'Neighborhood')   #selection of columns
    myframe1=callsbytype_attributes[A2].dropna() 
    if len(myframe1)>50:
        results[typeof]={}
        X=myframe1[regressors]
        Y=myframe1[typeof]
        #Random Forest 
        X_pre_train, X_test, label_pre_train, Y_test = train_test_split(X, Y, test_size=0.30, random_state=1)
        X_train, X_val, Y_train, Y_val = train_test_split(X_pre_train, label_pre_train, test_size=0.25, random_state=1)
        d=[]
        for mf,n_estim in itertools.product(max_features,n_estimators):
            forest=RandomForestRegressor(n_estimators=n_estim, max_depth=None, min_samples_split=2, min_samples_leaf=1,
             min_weight_fraction_leaf=0.0, max_features=mf,
             max_leaf_nodes=None, bootstrap=True, oob_score=False, n_jobs=1, random_state=None, verbose=0, warm_start=False)
            forest.fit(X_train,Y_train)
            forest_pred = forest.predict(X_val)
            name = '%s,%s'%(str(mf),str(n_estim))
            d.append((name,r2_score(Y_val,forest_pred)))
        best_solution=sorted(d, key=lambda tup: tup[1])[-1]
        parameters=best_solution[0].split(',')      
        mf=parameters[0]
        n_estim=parameters[1]
        forest=RandomForestRegressor(n_estimators=int(n_estim), max_depth=None, min_samples_split=2, min_samples_leaf=1,
         min_weight_fraction_leaf=0.0, max_features=mf,
         max_leaf_nodes=None, bootstrap=True, oob_score=False, n_jobs=1, random_state=None, verbose=0, warm_start=False)
        forest.fit(X_train,Y_train)
        results[typeof]['sample']=len(myframe1)
        ##############for getting a table with coefficients sorted by importance
        features_scores_list=sorted(zip(map(lambda x: round(x, 4), forest.feature_importances_), regressors), 
             reverse=True)
        results[typeof]['features']=pd.DataFrame(features_scores_list, columns=['slope_coeff', 'variable'])

In [ ]:
#the results are stored in the outputs folder, they can be seen at:
results_random_forest=pd.read_csv('outputs/rf_results_NTA.csv')